In [2]:
import pyabf
import matplotlib.pyplot as plt
import glob, sys, os, re
import numpy as np
import json
import pandas as pd
import seaborn as sns
import scipy.signal as ss        # Scientific package for signal processing
from pathlib import Path
from scipy.signal import find_peaks
from datetime import datetime
from scipy.ndimage import gaussian_filter1d
from PIL import Image
from tqdm.notebook import tqdm
from scipy import stats

In [3]:
df=pd.read_csv('recordings.csv', index_col=0)
df = df[(df['isValid']==1) & (df['protocol']=='current_clamp')]

In [4]:
df.head()

,Date,Line,age label,cell,uid,protocol,fname,dataLengthSec,sweepCount,cap,type,first_diff,age,age_group,isValid
17,2021-06-25 21:58:48.250,JNJ008 (diff 07062021),JNJ008 day 18,cell1 20pf,-3438561007472911673,current_clamp,data\JNJ008 (diff 07062021)\JNJ008 day 18\cell...,46.8,38,20.0,Control,2021-06-07 00:00:00.000,18,2-3 weeks,1.0
38,2021-07-22 11:41:16.023,JNJ008 (diff 07062021),JNJ008 day 45,cell1-35pf,-634694093113451121,current_clamp,data\JNJ008 (diff 07062021)\JNJ008 day 45\cell...,46.8,38,35.0,Control,2021-06-07 00:00:00.000,45,5-6 weeks,1.0
43,2021-07-22 11:56:41.876,JNJ008 (diff 07062021),JNJ008 day 45,cell2-32pf,5501609899512410291,current_clamp,data\JNJ008 (diff 07062021)\JNJ008 day 45\cell...,46.8,38,32.0,Control,2021-06-07 00:00:00.000,45,5-6 weeks,1.0
50,2021-07-22 12:16:11.466,JNJ008 (diff 07062021),JNJ008 day 45,cell3-42pf,5889560095625803792,current_clamp,data\JNJ008 (diff 07062021)\JNJ008 day 45\cell...,46.8,38,42.0,Control,2021-06-07 00:00:00.000,45,5-6 weeks,1.0
56,2021-07-22 13:11:05.785,JNJ008 (diff 07062021),JNJ008 day 45,cell4-33pf,-6276495275048037809,current_clamp,data\JNJ008 (diff 07062021)\JNJ008 day 45\cell...,46.8,38,33.0,Control,2021-06-07 00:00:00.000,45,5-6 weeks,1.0


In [5]:
set(df.Line)

{'JNJ008 (diff 07062021)',
 'JNJ008_PROX1 (diff 04112021)',
 'JNJ009 (diff 07062021)',
 'SND4652#2 (diff 15072021)',
 'SND4652#2_PROX1 (diff 15112021)'}

In [6]:
df.shape

(80, 15)

# AP peaks detection 

In [7]:
all_peak=[] # place holder to collect peaks indecis

for idx, row in tqdm(df.loc[:].iterrows(), total=df.shape[0]): 
    fig, (ax1, ax2) = plt.subplots(1,2,figsize=(40, 20))

    abf = pyabf.ABF(row['fname'])
    
    for sweep in abf.sweepList: #abf.sweepCount):

        abf.setSweep(sweep)   

        mask = np.zeros((abf.sweepPointCount-2,))
        ind_range = np.argwhere(abf.sweepC > 0).T[0][150:-100]
        mask[ind_range] = 1
        # mask = np.ones((abf.sweepPointCount-1,))

        x = abf.sweepY 
        xf = gaussian_filter1d(x,180)
        divY = np.diff(xf,n=2)  * (abf.dataRate / 1000) * mask # V/s
        
        # divPeaks, properties = find_peaks( divY , prominence=[10, None], )#, width=[5, None])
        divPeaks, properties = find_peaks(- 1000*divY ,  height=0.2,  width=[100, 800], prominence=[4, None], threshold=None, distance=100)

        peaks = []
        for dP in divPeaks: 
            prefix_offset = max(0,dP-200)
            peaksInWindow, properties = find_peaks( x[prefix_offset:dP+1000] , prominence=[1, None], width=[50, None])
            if (peaksInWindow.shape[0]>0): 
                if (prefix_offset + peaksInWindow[0] <14900) : # make sure to drop the peaks at the end-edge 
                    peaks.append(prefix_offset + peaksInWindow[0])
        
        for p in list(set(peaks)): 
            all_peak.append({"idx" : idx, 
                             "sweep": sweep, 
                             "evoked_AP": p})           
        
        offset = 5*sweep
        ax1.plot(x +offset)
        ax1.plot(peaks, x[peaks]+offset, "s", markersize =8)
        ax1.set_xlim(7000, 16000)
        
        # ax2.plot(xf +offset)
        # ax2.plot(peaks, xf[peaks]+offset, "s", markersize =8)
        # ax2.set_xlim(7000, 16000)
        
        ax2.plot(divY+offset/1000 , linestyle = '-')
        ax2.plot(divPeaks, divY[divPeaks]+offset/1000, "o", markersize =6)
        ax2.set_xlim(7000, 16000)
        

    ax1.set_title('idx: '+ str(idx) + '\n' + row['fname'])
    # ax2.set_title('Smooth signal')
    ax2.set_title('2nd derivative')
    plt.savefig(r'ap_peaks\{0}_{1}.png'.format(os.path.split(abf.abfFilePath)[-1][:-4], idx )) 
    plt.close()
    
    
pd.DataFrame(all_peak).to_json('evoked_AP_all_peaks.json')   


  0%|          | 0/80 [00:00<?, ?it/s]

## Peaks count

In [9]:
all_peak_df = pd.read_json('evoked_AP_all_peaks.json')

In [10]:
## Total_Evoked
for k, val in all_peak_df.groupby('idx').count()['evoked_AP'].iteritems(): 
    df.loc[k,'Total_Evoked'] = val

In [11]:
## max evoked of AP for each sweep and the AP threshold 
for idx,sub_df in all_peak_df.groupby('idx'): 
    max_ap=sub_df.groupby('sweep').count()['evoked_AP'].max()
    sweep_th = sub_df.groupby('sweep').count().sort_values('evoked_AP').iloc[0].name
    df.loc[idx,'max_Evoked'] = max_ap
    df.loc[idx,'sweep_threshold'] = sweep_th
    

In [12]:
df.to_csv('recordings_current_clamp.csv')

# Spike shape analysis 

In [13]:
def plot_peak_properties(x, label=None): 
    
    # first_peak_x, first_peak_properties = find_peaks(first_peak, prominence=[20, None], width=[20, None])
    
    peaks, properties = find_peaks(x, prominence=[1, None], width=[5, None])
    if len(peaks)>0: 
        fig, ax = plt.subplots(1,1)
        ax.plot(x)
        ax.plot(peaks, x[peaks], "x")
        ax.vlines(x=peaks, ymin=x[peaks] - properties["prominences"], ymax = x[peaks], color = "C1")
        ax.hlines(y=properties["width_heights"], xmin=properties["left_ips"], xmax=properties["right_ips"], color = "C1")
        ax.text(properties["right_ips"][0]+50, properties["width_heights"][-1], 'Height:{:.2f}\nWidth:{:.2f}'.format(properties['prominences'][0], properties['widths'][0]))

        if label: 
            fig.savefig(r'ap_peaks\shape_analysis\shape_{0}.png'.format(label)) 
            plt.close()

            # np.savetxt(r'ap_peaks\shape_analysis\shape_{0}.txt'.format(label),x, delimiter =", ")

        else: 
            plt.show()
        return properties

In [14]:
all_peak_df.head()

,idx,sweep,evoked_AP
0,17,9,9027
1,17,10,8500
2,17,11,8266
3,17,12,8210
4,17,13,8015


In [15]:
peak_shape_dict={}

for idx,sub_df in tqdm(all_peak_df.iloc[:].groupby('idx'), total=len(list(set(all_peak_df['idx'])))): 
    # load abf file
    abf = pyabf.ABF(df.loc[idx,'fname'])
    
    for k, row in sub_df.iterrows():  # for each sweep with peaks detected
        
        sweep = row['sweep']
        abf.setSweep(round(sweep))   
        x = abf.sweepY
        peak_idx = round(row['evoked_AP'])
        # take interval around detected peak, calculate papameters and save samples
        peak_interval =x[max(0,peak_idx-500):min(peak_idx+500, len(x))]
        peak_properties= plot_peak_properties(peak_interval,label=str((round(idx), round(sweep), round(row['evoked_AP']))))
        
        for key, value in peak_properties.items(): 
            all_peak_df.loc[k, key] = value[0]
            # peak_shape_dict[(idx, sweep, round(row['evoked_AP']))]= peak_interval
            np.savetxt(r'ap_peaks\shape_analysis\ap_shape_{}.txt'.format(str((idx, round(sweep), round(row['evoked_AP'])))),peak_interval, delimiter =", ")
            
            


  0%|          | 0/80 [00:00<?, ?it/s]

In [18]:
all_peak_df.head()

,idx,sweep,evoked_AP,prominences,left_bases,right_bases,widths,width_heights,left_ips,right_ips
0,17,9,9027,21.942139,0.0,968.0,285.050000,-11.367798,302.250000,587.300000
1,17,10,8500,34.576416,2.0,915.0,275.053571,-13.412476,329.875000,604.928571
2,17,11,8266,39.031982,0.0,906.0,245.354167,-12.863159,379.083333,624.437500
3,17,12,8210,35.308838,0.0,889.0,235.151786,-11.154175,402.062500,637.214286
4,17,13,8015,37.139893,0.0,886.0,213.888889,-8.468628,425.500000,639.388889


In [19]:
all_peak_df.to_csv('evoked_AP_all_peaks.csv')

In [ ]:
# import itertools

# a = ['Control', 'SLC1A4']
# b= ['widths', 'prominences']

# for c in itertools.product(a,b): 
#     mean= first_AP_df[first_AP_df['type']==c[0]][c[1]].mean()
#     sem = first_AP_df[first_AP_df['type']==c[0]][c[1]].sem()
    
#     print(f'{c[0]}, {c[1]} : {mean:.2f} ± {sem:.2f}')

In [20]:
# flist = list(Path('ap_peaks').glob('*.png'))
# FRAMES = [] # Empty list of frames
# for f in flist: 
#     im = Image.open(f)
#     FRAMES.append(im)
    
# FRAMES[0].save('ap_peaks/all.tiff', format="tiff", append_images=FRAMES[1:], save_all=True, duration=500, loop=0, compression='tiff_lzw',)    